# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.

## Imports

In [30]:
from dotenv import load_dotenv
from os import environ as ENV

import psycopg2
import pandas as pd
import altair as alt

## Setup

In [31]:
load_dotenv()

True

In [32]:
conn = psycopg2.connect(user=ENV["DATABASE_USERNAME"],
                        password=ENV["DATABASE_PASSWORD"],
                        host=ENV["DATABASE_IP"],
                        port=ENV["DATABASE_PORT"],
                        dbname=ENV["DATABASE_NAME"])

## Load Data

In [33]:
with conn.cursor() as curs:
    query = ("""
            SELECT * FROM exhibition;
            """)
    exh = pd.read_sql_query(query, conn)
    

/var/folders/rd/sh7_m8ds32lgl8y5rl2qnr_40000gn/T/ipykernel_16414/1300187427.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exh = pd.read_sql_query(query, conn)


In [34]:
with conn.cursor() as curs:
    query = ("""
             SELECT * FROM exhibition
             LEFT JOIN request_interaction
             USING (exhibition_id)
             LEFT JOIN rating_interaction
             USING (exhibition_id);
            """)
    exh_incident = pd.read_sql_query(query, conn)

/var/folders/rd/sh7_m8ds32lgl8y5rl2qnr_40000gn/T/ipykernel_16414/783772945.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exh_incident = pd.read_sql_query(query, conn)


In [35]:
with conn.cursor() as curs:
    query = ("""
             SELECT * FROM rating_interaction
             LEFT JOIN rating
             USING (rating_id);
            """)
    rating = pd.read_sql_query(query, conn)

/var/folders/rd/sh7_m8ds32lgl8y5rl2qnr_40000gn/T/ipykernel_16414/3342658643.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rating = pd.read_sql_query(query, conn)


In [36]:
with conn.cursor() as curs:
    query = ("""
             SELECT * FROM request_interaction
             LEFT JOIN request
             USING (request_id);
            """)
    requests = pd.read_sql_query(query, conn)

/var/folders/rd/sh7_m8ds32lgl8y5rl2qnr_40000gn/T/ipykernel_16414/4179121845.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  requests = pd.read_sql_query(query, conn)


In [37]:
with conn.cursor() as curs:
    query = ("""
             SELECT * FROM exhibition
             LEFT JOIN department
             USING (department_id)
             LEFT JOIN rating_interaction
             USING (exhibition_id)
             LEFT JOIN rating
             USING (rating_id);
            """)
    exh_dep = pd.read_sql_query(query, conn)

/var/folders/rd/sh7_m8ds32lgl8y5rl2qnr_40000gn/T/ipykernel_16414/3575413940.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exh_dep = pd.read_sql_query(query, conn)


## Exploration

### Task 1: What exhibition is most frequently visited

In [38]:
exh_incident['exhibition_id'].value_counts()

exhibition_id
6    13630
4     5362
5     5291
1     4230
2     2709
3        1
Name: count, dtype: int64

In [39]:
exh[exh['exhibition_id'] == 6]

,exhibition_id,exhibition_name,exhibition_description,department_id,floor_id,exhibition_start_date,public_id
5,6,Cetacean Sensations,Whales: from ancient myth to critically endang...,4,2,2019-07-01,EXH_03


### Task 2: What hour of the day has the most ratings

In [40]:
rating.head()

,rating_id,rating_interaction_id,exhibition_id,event_at,rating_value,rating_description
0,1,1,5,2023-03-06 15:09:21+00:00,0,Terrible
1,2,2,1,2023-03-06 17:43:27+00:00,1,Bad
2,4,3,1,2023-03-06 13:13:34+00:00,3,Good
3,4,4,2,2023-03-06 17:20:43+00:00,3,Good
4,2,5,5,2023-03-06 14:54:21+00:00,1,Bad


In [41]:
rating['event_at'].dt.hour.value_counts().head()

event_at
10    232
13    228
17    225
14    223
9     221
Name: count, dtype: int64

### Task 3: What exhibition has the most emergencies?

In [42]:
requests.head()

,request_id,request_interaction_id,exhibition_id,event_at,request_value,request_description
0,1,1,1,2023-03-06 17:12:19+00:00,0,assistance
1,1,2,6,2023-03-06 14:26:42+00:00,0,assistance
2,1,3,4,2023-03-06 14:06:33+00:00,0,assistance
3,1,4,5,2023-03-06 12:56:44+00:00,0,assistance
4,1,5,4,2023-03-06 18:06:47+00:00,0,assistance


In [43]:
requests.groupby(by='exhibition_id')['request_description'].value_counts()

exhibition_id  request_description
1              assistance             10
2              assistance              9
4              assistance             12
               emergency               2
5              assistance             11
6              assistance             29
Name: count, dtype: int64

In [44]:
exh[exh['exhibition_id'] == 4]

,exhibition_id,exhibition_name,exhibition_description,department_id,floor_id,exhibition_start_date,public_id
3,4,The Crenshaw Collection,"An exhibition of 18th Century watercolours, mo...",4,3,2021-03-03,EXH_02


### Task 4: What is the average rating for each exhibition?

In [45]:
rating

,rating_id,rating_interaction_id,exhibition_id,event_at,rating_value,rating_description
0,1,1,5,2023-03-06 15:09:21+00:00,0,Terrible
1,2,2,1,2023-03-06 17:43:27+00:00,1,Bad
2,4,3,1,2023-03-06 13:13:34+00:00,3,Good
3,4,4,2,2023-03-06 17:20:43+00:00,3,Good
4,2,5,5,2023-03-06 14:54:21+00:00,1,Bad
...,...,...,...,...,...,...
2053,1,2054,4,2023-03-06 16:10:31+00:00,0,Terrible
2054,3,2055,5,2023-03-06 16:10:05+00:00,2,Neutral
2055,4,2056,1,2023-03-06 14:27:52+00:00,3,Good
2056,4,2057,6,2023-03-06 10:41:17+00:00,3,Good


In [46]:
rating.groupby('exhibition_id')['rating_value'].mean()

exhibition_id
1    1.929078
2    1.916944
4    1.428198
5    1.216216
6    2.825532
Name: rating_value, dtype: float64

In [47]:
exh[exh["exhibition_id"] == 6] 

,exhibition_id,exhibition_name,exhibition_description,department_id,floor_id,exhibition_start_date,public_id
5,6,Cetacean Sensations,Whales: from ancient myth to critically endang...,4,2,2019-07-01,EXH_03


### Task 5: Are positive ratings more frequent before or after 1pm

In [48]:
bad_ratings = rating[(rating['rating_description'] == "Terrible") 
                     | (rating['rating_description'] == "Bad")]

In [49]:
good_ratings = rating[(rating['rating_description'] == "Good")
                     | (rating['rating_description'] == "Amazing")]

In [50]:
bad_ratings.groupby(by=rating['event_at'].dt.hour <= 13)['rating_description'].count()

event_at
False    407
True     443
Name: rating_description, dtype: int64

- True is before 1pm, False is after so the positive events are more frequent before 1pm (inclusive)

In [51]:
good_ratings.groupby(by=rating['event_at'].dt.hour <= 13)[
    'rating_description'].count()

event_at
False    272
True     383
Name: rating_description, dtype: int64

### Task 6: Do Zoology exhibitions get better ratings than other types

In [52]:
exh_dep

,rating_id,exhibition_id,department_id,exhibition_name,exhibition_description,floor_id,exhibition_start_date,public_id,department_name,rating_interaction_id,event_at,rating_value,rating_description
0,1.0,5,5,Our Polluted World,"A hard-hitting exploration of humanity""s impac...",4,2021-05-12,EXH_04,Ecology,1.0,2023-03-06 15:09:21+00:00,0.0,Terrible
1,2.0,1,1,Adaptation,How insect evolution has kept pace with an ind...,1,2019-07-01,EXH_01,Entomology,2.0,2023-03-06 17:43:27+00:00,1.0,Bad
2,4.0,1,1,Adaptation,How insect evolution has kept pace with an ind...,1,2019-07-01,EXH_01,Entomology,3.0,2023-03-06 13:13:34+00:00,3.0,Good
3,4.0,2,2,Measureless to Man,An immersive 3D experience: delve deep into a ...,2,2021-08-23,EXH_00,Geology,4.0,2023-03-06 17:20:43+00:00,3.0,Good
4,2.0,5,5,Our Polluted World,"A hard-hitting exploration of humanity""s impac...",4,2021-05-12,EXH_04,Ecology,5.0,2023-03-06 14:54:21+00:00,1.0,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,3.0,5,5,Our Polluted World,"A hard-hitting exploration of humanity""s impac...",4,2021-05-12,EXH_04,Ecology,2055.0,2023-03-06 16:10:05+00:00,2.0,Neutral
2055,4.0,1,1,Adaptation,How insect evolution has kept pace with an ind...,1,2019-07-01,EXH_01,Entomology,2056.0,2023-03-06 14:27:52+00:00,3.0,Good
2056,4.0,6,4,Cetacean Sensations,Whales: from ancient myth to critically endang...,2,2019-07-01,EXH_03,Zoology,2057.0,2023-03-06 10:41:17+00:00,3.0,Good
2057,3.0,6,4,Cetacean Sensations,Whales: from ancient myth to critically endang...,2,2019-07-01,EXH_03,Zoology,2058.0,2023-03-06 17:42:48+00:00,2.0,Neutral


In [53]:
exh_dep['department_name'].value_counts()

department_name
Zoology         853
Ecology         481
Entomology      423
Geology         301
Paleontology      1
Name: count, dtype: int64

In [54]:
exh_dep.groupby(by='department_name')['rating_value'].mean()

department_name
Ecology         1.216216
Entomology      1.929078
Geology         1.916944
Paleontology         NaN
Zoology         2.198124
Name: rating_value, dtype: float64

In [55]:
department_ratings = exh_dep.groupby(by='department_name')['rating_value'].value_counts().reset_index()

In [56]:
department_ratings.head()

,department_name,rating_value,count
0,Ecology,0.0,166
1,Ecology,1.0,142
2,Ecology,2.0,113
3,Ecology,4.0,37
4,Ecology,3.0,23


In [57]:
alt.Chart(department_ratings).mark_bar().encode(
    x='rating_value:N',
    y='count:Q',
    color='rating_value',
    column='department_name'
)

alt.Chart(...)